# Purpose

The purpose of this application is to publish the YAML formatted document as HTML, PDF, and CommonMark

# References

Put useful stuff here

* [PyYAML](https://pyyaml.org/)
* [pandoc](https://pandoc.org/)
  * [TeX Live](https://www.tug.org/texlive/) which is needed to [create a PDF](https://pandoc.org/MANUAL.html#creating-a-pdf)

# Install

Anything that needs installed to use this should be listed here

``` shell
#PyYAML
sudo -H pip3 install pyyaml

#Pandoc
sudo apt install pandoc
sudo apt install texlive-latex-recommended
```

# Application

## Configuration

In [ ]:
srcDir="./src"
dstDir="./docs"

#extensions to use for the various output types
#use: `extentions['yaml']` to receive the value
extentions = {
    'yaml': '.yaml',
    'pandoc': '.pandoc',
    'markdown': '.md',
    'html': '.html',
    'pdf': '.pdf'
}

## function definitions

### find()

In [35]:
#find files that match the pattern

#these functions came from:
#https://stackoverflow.com/questions/1724693/find-a-file-in-python

"""
#this will find the first match
import os
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

#this will find all matches
import os
def find_all(name, path):
    result = []
    for root, dirs, files in os.walk(path):
        if name in files:
            result.append(os.path.join(root, name))
    return result
"""

#this will match a pattern
import os, fnmatch
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result
#find('*.txt', '/path/to/dir')

### mkdir_p() and safe_open_w()

In [40]:
#the following is from:
#https://stackoverflow.com/questions/23793987/write-file-to-a-directory-that-doesnt-exist

import os, os.path
import errno

# Taken from https://stackoverflow.com/a/600612/119527
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc: # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else: raise

def safe_open_w(path):
    ''' Open "path" for writing, creating any parent directories as needed.
    '''
    mkdir_p(os.path.dirname(path))
    return open(path, 'w')

#with safe_open_w('/Users/bill/output/output-text.txt') as f:
#    f.write(...)

### yaml2python()

In [36]:
#this will convert a yaml file to a python data structure.

import yaml

def yaml2python (file):
    #print (file)

    f = open (file, 'r') #open the file to read
    content = yaml.load(f.read(), Loader=yaml.FullLoader) #convert the yaml contents into a python data structure
    f.close() #close the file
    
    #print (content)
    
    return content

### generatePandocFile()

In [37]:
#`data` = the python data structure object
#`output` = the output file

def generatePandocFile (data, output):
    #open a file for output
    #f = open(output, 'w')
    f = safe_open_w(output)

    #Pandoc’s Markdown [Metadata blocks](https://pandoc.org/MANUAL.html#metadata-blocks)
    #Extension: yaml_metadata_block
    #
    #\n
    #title:
    #author:
    #keywords:
    #abstract:
    #
    #print ('\n')
    #print ('title: "', data['title'], '"', file=f)
    #print ('author: "', data['author'], '"', file=f)
    #print ('% ', data['date'], file=f)
    #print ('"subtitle": "', data['subtitle'], '"', file=f)
    #print ('"description": "', data['description'], '"', file=f)
    #print ('"lang": "', data['lang'], '"', file=f)
    
    
    
    
    #print ("\n---\n", file=f)

    print (data['documentBody'], file=f)

    
    print ("\n---\n", file=f)

    
    print ("# Revision history", file=f)
    print ("\n")
    
    #print the revisions as a table in HTML because CommonMark doesnt do that

    #table style
    print ("<style>", file=f)
    print ("table { border-collapse: collapse; width: 100%; }", file=f)
    print ("td, th { border: 1px solid #dddddd; text-align: left; padding: 8px; }", file=f)
    print ("</style>", file=f)
    print ("\n")
    
    #Table layout:  "date", "status", "name", "reason"
    print ("<table>", file=f)
    print ("<tr>", file=f)
    print ("<th>Date</th>", file=f)
    print ("<th>Name</th>", file=f)
    print ("<th>Reason</th>", file=f)
    print ("</tr>", file=f)
    #the 'revisions' key consists of a list of sets
    #`content['revision'][0]['date']` will provide that specific data element
    for element in data['revision']:  #step through the list elements
        print ("<tr>", file=f)
        #now print the fields
        print ("<td>",element['date'],"</td>", file=f)
        print ("<td>",element['name'],"</td>", file=f)
        print ("<td>",element['reason'],"</td>", file=f)
        print ("</tr>", file=f)
    print ("</table>", file=f)

    f.close()  #close the file

### convert2md(), convert2html(), convert2pdf

In [ ]:
#convert the pandoc file to MarkDown
import subprocess

def convert2md():
    subprocess.run(['pandoc', content['id']+'.pandoc', '--html-q-tags', '-s', '-o', content['id']+'.md'])

In [ ]:
#convert the pandoc file to html
import subprocess

def convert2html():
    subprocess.run(['pandoc', content['id']+'.pandoc', '--html-q-tags', '-s', '-o', content['id']+'.html'])

the pdf conversion is generating errors

In [ ]:
#convert the pandoc file to pdf
import subprocess

def convert2pdf():
    subprocess.run(['pandoc', content['id']+'.pandoc', "-o", content['id']+'.pdf'])

## main program

### find the files that need to be processed

In [33]:
#look for all the yaml files

srcFiles = find('*'+extentions['yaml'], srcDir)
#srcFiles

### process each file

In [32]:
import yaml

for file in srcFiles:
    print (file)

    content = yaml2python (file)
    #print (content)
    
    #for item, value in content.items():
    #    print(item, ":", value)
    
    #outputFile=dstDir+"/"+content['id']+"/"content['id']+extentions['pandoc'] #construct the full destination file/path
    #generatePandocFile (data, outputFile)
    
    #convert2html()
    
    #save data that will be used for the summary

./src/32f2e7dc-0d00-4bd9-b39f-9d5b562f8415.yaml
./src/ad9c7149-fa91-4e62-a86b-fdc4387840d1.yaml
./src/1e7c647e-93d7-455d-b5a5-fd7205ca1b14.yaml
./src/cd498dae-9287-4432-91a3-97adf1ea4dd6.yaml
./src/c54c285f-eeb4-4a42-815f-9ea0656265e2.yaml
./src/ebd4a2a1-1c57-4dfb-b13b-c02355a40d74.yaml
./src/4480c89d-1da0-4f18-9f8b-f0238333e69a.yaml
./src/93a0be83-8e5a-4a46-b264-218646b412ce.yaml
./src/4a4d6ec7-4cce-45f8-aa79-df1764058818/4a4d6ec7-4cce-45f8-aa79-df1764058818.yaml


### generate summary file